<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Differential_Privacy_in_GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define a simple neural network model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, optimizer, and loss function
model = SimpleModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Define a function to add DP noise to gradients
def add_dp_noise(grad, noise_scale=1e-5):
    noise = torch.normal(0, noise_scale, grad.shape).to(grad.device)
    return grad + noise

# Transform and DataLoader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Training loop with differential privacy
n_epochs = 10
for epoch in range(n_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        for param in model.parameters():
            if param.grad is not None:
                param.grad = add_dp_noise(param.grad)  # Add DP noise to gradients
        optimizer.step()

    print(f"Epoch [{epoch+1}/{n_epochs}] Loss: {loss.item()}")

print("Training with differential privacy completed!")